In [1]:
import os
import torch
from torch import nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

if torch.cuda.is_available():
    device = "cuda" #NVIDI A GPU
elif torch.backends.mps.is_available():
    device = "mps" #Apple GPU
else:
    device = "cpu"




In [2]:
class getCarFollowSpeedModel(nn.Module):
    def __init__(self):
        super().__init__()
        #self.flatten = nn.Flatten()

        self.relu_stack = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),

            #hidden layers
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),

            nn.Linear(64, 1),
        )

    def forward(self, x):
        speed = self.relu_stack(x)
        return speed




In [3]:
model = getCarFollowSpeedModel()

#testing
test_data = torch.rand(10)
with torch.inference_mode():
    y = model(test_data)

test_data

tensor([0.8899, 0.1299, 0.7413, 0.8745, 0.5060, 0.6960, 0.5091, 0.3573, 0.2686,
        0.2862])

#Training with a simple dataset

1.Front vehicle speed                C++: leader->getSpeed()      Python API: get_leader(vehicle_id), get_vehicle_info(vehicle_id)

2.Front vehicle max dcceleration     C++: leader->getMaxNegAcc()  Python API: No API. The default value is 4.5

3.target vehicle's own speed         C++: vehicleInfo.speed       Python API: get_vehicle_speed() or get_vehicle_info(vehicle_id)

4.target vehicle's max deceleration  C++: vehicleInfo.maxNegAcc   Python API: No API, The default value is 4.5

5.interval between cars in the traffic flow   C++: interval       Python API: No API, The default value is 1.0

6.gap between target vehicle and front vehicle  C++: controlInfo.gap  Python API: No Api, let's just use 2.5 for now

7.front vehicle's usual deceleration     C++: leader->getUsualNegAcc() Python API: No API, The default value is 2.5

8.target vehicle's usual deceleartion    C++: vehicleInfo.usualNegAcc   Python API: No API, The default value is 2.5    

9.minimum gap allowed between target vehicle and front vehicle     C++:vehicleInfo.minGap   Python API: The default is 2.5

10.target vehicle's headway time    C++: vehicleInfo.headwayTime    The defualt value is 1.5

In [4]:
list(model.parameters())

[Parameter containing:
 tensor([[-1.4135e-01, -1.4084e-01,  9.8059e-02,  2.4603e-01, -5.0181e-02,
           9.8987e-02, -1.4176e-02, -1.1133e-01, -1.2341e-01,  1.2611e-01],
         [ 2.5408e-01,  2.2845e-01, -2.7732e-01,  1.0951e-01,  1.6268e-01,
           1.9148e-01,  3.0052e-01,  1.3140e-01,  2.9943e-01,  3.0787e-01],
         [-2.6603e-01,  3.0962e-01,  2.5486e-01,  3.0655e-01,  2.0019e-01,
           8.0447e-02, -1.0877e-01, -2.5287e-01, -7.3381e-02,  1.0496e-01],
         [-1.8430e-01,  1.4436e-01,  1.7900e-01,  1.0573e-01, -2.6206e-02,
           2.3302e-01, -6.9523e-02, -2.4988e-01,  2.7891e-01,  6.5189e-02],
         [ 1.7585e-01,  6.0722e-02, -4.0635e-02, -2.8840e-01, -1.1596e-01,
           1.1740e-01,  2.7922e-01, -1.4156e-01,  1.1997e-01,  1.1105e-02],
         [ 1.4394e-01, -4.6715e-02,  2.7981e-01, -2.6077e-01, -3.1681e-02,
           1.3301e-01, -2.4534e-01,  2.6045e-02, -2.8544e-01,  9.6482e-02],
         [-8.0280e-02,  7.9783e-02,  3.0369e-01, -1.5661e-01,  8.5215e-

In [7]:
# a simple set of training data 
#traget vehicle: flow_7_1
#front vehicle: flow_7_0
leader_speed = 16.670000
leader_max_deccel = 4.5
target_current_speed = 10.0
target_max_deccel = 4.5
interval = 1.0
gap = 2.5
leader_usual_deceleration = 2.5
target_usual_deceleration = 2.5
min_gap = 2.5
target_headway = 1.5

tagrget_next_speed = 12.0
tagrget_next_speed = torch.tensor(tagrget_next_speed, dtype=torch.float32)


epochs = 100

#loss function
loss_fn = nn.L1Loss()
#optimizer
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)


for epoch in range(epochs):
    model.train()
    pred_speed = model(torch.tensor([leader_speed, leader_max_deccel, target_current_speed, target_max_deccel, interval, gap, leader_usual_deceleration, 
                                    target_usual_deceleration, min_gap, target_headway]))
   
    loss = loss_fn(pred_speed, tagrget_next_speed)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    #testing while training

    if epoch % 10 == 0:
        print(loss)

model.eval()


tensor(11.2849, grad_fn=<MeanBackward0>)
tensor(11.0965, grad_fn=<MeanBackward0>)
tensor(10.8435, grad_fn=<MeanBackward0>)
tensor(10.4879, grad_fn=<MeanBackward0>)
tensor(9.9669, grad_fn=<MeanBackward0>)
tensor(9.1518, grad_fn=<MeanBackward0>)
tensor(7.7315, grad_fn=<MeanBackward0>)
tensor(4.8620, grad_fn=<MeanBackward0>)
tensor(0.2635, grad_fn=<MeanBackward0>)
tensor(0.0436, grad_fn=<MeanBackward0>)


getCarFollowSpeedModel(
  (relu_stack): Sequential(
    (0): Linear(in_features=10, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [6]:
#save as torch script
model_scripted = torch.jit.script(model)
model_scripted.save('model_scripted.pt')